In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import os
from osgeo import gdal
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
batch_size = 5
num_classes = 2
epochs = 5

img_rows, img_cols = 81, 81

array = []
et = []

In [3]:
def baseline_model():
    model = Sequential()

    model.add(Conv2D(32, (2, 2), input_shape=(81,81,1)))
    model.add(Activation('relu'))
    BatchNormalization(axis=-1)
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, (2, 2)))
    model.add(MaxPooling2D(pool_size=(2,2)))
 
 
    BatchNormalization(axis=-1)
    model.add(Conv2D(64,(3, 3)))
    model.add(Activation('relu'))
    BatchNormalization(axis=-1)
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
# 

    model.add(Flatten())
#    # Fully connected layer
#
    BatchNormalization()
    model.add(Dense(512))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.5))
    model.add(Dense(2))
    
    sgd = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',#keras.optimizers.Adam(),
              metrics=['accuracy'])
    
    return model

In [22]:
model = baseline_model()
model.load_weights("weights.best94.hdf5")

In [11]:
x_train = np.load("x.npy")
y_train = np.load("y.npy")
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
y_train = keras.utils.to_categorical(y_train, num_classes)

In [23]:
y_pred = model.predict(x_train)
for i in range(len(y_pred)):
    if y_pred[i][0] < y_pred[i][1]:
        y_pred[i]=[0,1]
    else:
        y_pred[i]=[1,0]

In [24]:
yy_test = []
yy_pred = []
for i in range(len(y_train)):
    if y_pred[i][0] == 0:
        yy_pred.append(1)
    else:
        yy_pred.append(0)
    
    if y_train[i][0] == 0:
        yy_test.append(1)
    else:
        yy_test.append(0)

In [25]:
confusion_matrix(yy_test, yy_pred)

array([[1707,  165],
       [  64, 1808]], dtype=int64)

In [29]:
from sklearn.metrics import recall_score,precision_score
print(recall_score(yy_test,yy_pred))
print(precision_score(yy_test,yy_pred))

0.965811965812
0.916371008616


In [21]:
model.save('best_ever.h5')